# Introduction
This notebook demonstrate a machine learning model project prototyping from [Grab AI Challengle in Traffic Management](https://www.aiforsea.com/traffic-management) as a starting kit.

## Objective

> Economies in Southeast Asia are turning to AI to solve traffic congestion, which hinders mobility and economic growth. The first step in the push towards alleviating traffic congestion is to understand travel demand and travel patterns within the city.
> 
> Can we accurately forecast travel demand based on historical Grab bookings to predict areas and times with high travel demand?

In [ ]:
!pip install pendulum

In [ ]:
!pip install fbprophet

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import Geohash
from matplotlib import pyplot as plt
from math import sin, cos, sqrt, atan2, radians
import pendulum
from scipy.interpolate import interpn
from sklearn.preprocessing import LabelBinarizer , LabelEncoder, OneHotEncoder, MinMaxScaler
import os
print(os.listdir("../input"))

from numpy import split
from numpy import array
from sklearn.metrics import mean_squared_error

import utils
import gc # garbage collector
import seaborn as sns
% matplotlib inline
plt.style.use('seaborn-whitegrid')

In [ ]:
import fbprophet
fbprophet.__version__

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM
from keras.layers import RepeatVector
from keras.layers import TimeDistributed

In [ ]:
import plotly
plotly.__version__
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

# Data Exploratory and wrangling
First step in machine learning is always about understanding the data

In [ ]:
df = pd.read_csv("../input/training.csv")

In [ ]:
# check for null values
df[df.isnull().any(axis=1)]

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.head()

### Decoding geohash6
We will do a decoding into latitude and longitude back this encoding to find out more information about the the data location.
> Geohash is a public domain geocoding system invented by Gustavo Niemeyer[1], which encodes a geographic location into a short string of letters and digits. It is a hierarchical spatial data structure which subdivides space into buckets of grid shape, which is one of the many applications of what is known as a Z-order curve, and generally space-filling curves (source: wikipedia).

More information about the geohash can be found in wikipedia: [wikipedia](https://en.wikipedia.org/wiki/Geohash): 

> So geohash6 means that it has 6 characters with the precision of ±0.61km (610m). I suspect landmark or point of interest is already compressed into this single encoding value.

In [ ]:
# get the unique hashes
geohashes_df = df.groupby('geohash6', as_index=False)\
.agg({'day':'count'})\
.rename(columns={'day':'count'})\
.sort_values(by='count', ascending=False)

In [ ]:
print("Total encoded hash",len(geohashes_df))

In [ ]:
geohashes_df.sort_values(by='count', ascending=False).head()

In [ ]:
# find out the distribution of the hash
ax = sns.distplot(geohashes_df['count'], rug=True, hist=False)

In [ ]:
%%time
# decode geohash into lat and long
geohashes_df['lat'] = None
geohashes_df['lat_err'] = None
geohashes_df['long'] = None
geohashes_df['long_err'] = None
for i in range(len(geohashes_df)):
    geo_decoded = Geohash.decode_exactly(geohashes_df.loc[i,'geohash6'])
    geohashes_df.loc[i,'lat'] = geo_decoded[0]
    geohashes_df.loc[i,'long'] = geo_decoded[1]
    geohashes_df.loc[i,'lat_err'] = geo_decoded[2]
    geohashes_df.loc[i,'long_err'] = geo_decoded[3]

In [ ]:
# top 10 geo
geohashes_df.head(10)

In [ ]:
# merge into original df
df = df.merge(geohashes_df.drop(columns=['count']), on='geohash6', how='inner')

In [ ]:
# convert lat and long into float type
df['lat'] = df['lat'].astype('float64')
df['long'] = df['long'].astype('float64')
df['lat_err'] = df['lat_err'].astype('float64')
df['long_err'] = df['long_err'].astype('float64')

### Timestamp and day
The timestamp is not a linux timestamp but rather a `hour:minute` form. Another thing we don't have is the context of  month and year. But we'll make an attempt to deduce. Knowing which month and year could help us in forming seasonality or holiday that potentially affecting the prediction

In [ ]:
# extract hour and minute from timestamp column
df[['h','m']] = df['timestamp'].str.split(':',expand=True)
df['h'] = df['h'].astype('int64')
df['m'] = df['m'].astype('int64')

In [ ]:
# extract day of week (DoW) from day
# since we have no idea about which month is this data were taken so can't
# be sure the DoW starts on which day
# but it's good enought we dont need to dig deeper I supposed
# but in test set clearly this thing needs to be mapped out correcly
df['dow'] = df['day'] % 7

In [ ]:
df.head(1)

In [ ]:
# outlier from day
zscore = lambda x: (x - x.mean()) / x.std()
df['zscore_day'] = np.abs(df.groupby('day')['demand'].transform(zscore))
print("number of suspected outliers from day", len(df[df['zscore_day'] > 3]))

In [ ]:
# outlier from timestamp
df['zscore_timestamp'] = np.abs(df.groupby('timestamp')['demand'].transform(zscore))
print("number of suspected outliers from timestamp", len(df[df['zscore_timestamp'] > 3]))

In [ ]:
_ = df[(df['zscore_day'] <= 3) & (df['zscore_timestamp'] <= 3)].groupby(['dow','h'], as_index=False)\
.agg({'demand':'mean'})\
.sort_values(by=['dow','h'])

In [ ]:
_.head()

In [161]:
data = [
    go.Heatmap(
        z=_['demand'].values.reshape((7,24)),
        x=[str(i) for i in range(24)],
        y=[str(i) for i in range(7)],
        #colorscale='Viridis',
    )
]

layout = go.Layout(
    title='Average Demand in Day of Week x 24 hours',
    xaxis=dict(
        title='Hours',
    ),
    yaxis=dict(
        title='Week of Day',
    ),
)

fig = go.Figure(data=data, layout=layout)
iplot(fig)

### Heatmap analysis
* We have no context which month the day 0 is
* But from heatmap I would speculate that the 0 is Monday 
* Since we can observe the stripe on 6th DoW noticably deviating from the rest, this could indicate Sunday
* Looking at the "First month" vs "second month" plot, I suspect the dataset were derived from Malaysia and for October and November 2018.

In [ ]:
# let's find out since 2018 which month has monday for their first day
start = pendulum.datetime(2018, 1, 1)
end = pendulum.datetime(2019, 5, 22)

period = pendulum.period(start, end)

for dt in period.range('days'):
    if dt.day_of_week == pendulum.MONDAY and dt.day == 1:
        print(dt.to_date_string(), "number of days:", dt.end_of('month').day-dt.start_of('month').day+1)

In [ ]:
_ = df.groupby('day', as_index=False).agg({'demand':'mean'})

In [ ]:
x = [i for i in range(31)]
trace1 = go.Scatter(
    x = x,
    y = _['demand'][0:31],
    mode = 'lines+markers',
    name = 'First Month'
)
trace2 = go.Scatter(
    x = x,
    y = _['demand'][31:61],
    mode = 'lines+markers',
    name = 'Second Month'
)

data = [trace1, trace2]
iplot(data)

In [ ]:
# demand distribution
df['demand'].hist(bins=100, figsize=(14,3))
plt.xlabel('Demand normalized')
plt.title('Histogram');

In [ ]:
df.corr()

### Analysis of the Geo data

In [ ]:
def calc_dist(lat1, long1, lat2, long2):
    # calculate distance between two coordinate using Haversine formula
    # approximate radius of earth in km
    R = 6373.0
    lat1 = radians(lat1)
    lon1 = radians(long1)
    lat2 = radians(lat2)
    lon2 = radians(long2)
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance

In [ ]:
# minimum and maximum latitude
lat_min, lat_max = df['lat'].min(), df['lat'].max()
# minimum and maximum longitude
long_min, long_max = df['long'].min(), df['long'].max()

In [ ]:
print("Min coordinate", (lat_min, long_min))
print("Max coordinate", (lat_max, long_max))

In [ ]:
lat_A = lat_max
long_A = long_min
lat_B = lat_min
long_B = long_max
dA = calc_dist(lat_min, long_min, lat_A, long_A)
dB = calc_dist(lat_min, long_min, lat_B, long_B)
diameter = calc_dist(lat_min, long_min, lat_max, long_max)

### Calculate the bounding area
* The square area (KM) of the GPS bounding box is $ 1,170 km^2 $ . So I imagine the size close to Penang

In [ ]:
print("Circle Area",np.pi * (diameter/2)**2)
print("Square area", dA*dB)

## Plot into Map
Here we want to understand if we can use external data augmentation from the geo location. But turns out the location were masked into ocean. Unless grab is providing on demand hailing to Atlantist, I'm afraid we can't do much use the geo data for augmentation such as landmarks (point of interest).

In [ ]:
# https://www.openstreetmap.org/#map=4/-6.61/109.16
# The bounding box in openstreetmap: left, bottom, right, top (min long, min lat, max long, max lat)
bbox = (89.00, 119.31,-9.15, 10.17) # we'll plot long on x-axis
sea_map = plt.imread("http://madet.my/images/map_sea_trim.png")

In [ ]:
alpha=0.3
s=1
fig, ax = plt.subplots(1, 1, figsize=(16,10))
ax.scatter(df['long'],df['lat'], zorder=1, alpha=alpha, c='r', s=s)
ax.set_xlim((bbox[0], bbox[1]))
ax.set_ylim((bbox[2], bbox[3]))
ax.set_title('SEA Map (partial) with scatter points of demand geolocation')
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")
ax.imshow(sea_map, zorder=0, extent=bbox)
plt.show()

In [ ]:
%%time
x = df['lat'].values
y = df['long'].values
plt.figure(figsize=(14,8))
data , x_e, y_e = np.histogram2d( x, y)
z = interpn( ( 0.5*(x_e[1:] + x_e[:-1]) , 0.5*(y_e[1:]+y_e[:-1]) ) , 
            data , np.vstack([x,y]).T , method = "splinef2d", bounds_error = False)
idx = z.argsort()
x, y, z = x[idx], y[idx], z[idx]
plt.scatter(x, y, c=z, cmap='magma')
plt.colorbar()
plt.xlabel('Latitude')
plt.ylabel('Longitude')
plt.title('Density of demand by location');

# Baseline Models
* This is a supervised learning problem where we can throw to the model the features as well as actual values to be learned. 
* Since we have multiple features to be used as forecasting step, it's fall under multivariate forecasting
* We'll tackle using 2 simple approach here: 
  * Single variate time series forecasting and use parallel training (@todo)
  * fbprohet multivariate through regressor
  * Neural network
  
  
* Here what I meant by baseline model is that there is no improvement after first time running the model. In the actual machine learning project one would usually have this similar approach and incrementally fine tune the model to produce better result such as finding best hyperparameter, adding more features, remove outliers, and etc
* From my experience working with machine learning project major challenge usualy is overfit and imbalance class issues. If we look at the the geohash observation some only got few example so it could potentially overfit and produce far incorrect prediction in production environment. Another thing to observe is outliers, one scenario is sudden spike in certain areas due to a random events. We probably want to exclude this if such spike is not repeatable in seasonality pattern.
  

In [ ]:
# label encode the geohash6
# though the best way is probably onehot encode, but we'll leave it to next iteration
labelencoder = LabelEncoder()
df['geo_encoded'] = labelencoder.fit_transform(df['geohash6'])

In [ ]:
# @TODO encode the geo to onehot or binary
# lb = LabelBinarizer(sparse_output=False)
# lb.fit(df['geo_encoded'])
# we'll transform geohash6 into onehot
# lb.transform([212]).shape

In [ ]:
# select only column we're interested in
df = df[['day','h', 'm', 'dow','geo_encoded','demand']]

## Fbprophet
fbprophet is the easiest tool for forecasting where you can model your solution quickly and sometime it's already good enough.

In [ ]:
Prophet = fbprophet.Prophet

In [ ]:
df_copy = df.copy()
df_copy['year'] = 2018
df_copy['day_'] = df_copy['day'].apply(lambda x : x if x <= 31 else x-31 )
df_copy['month'] = df_copy['day'].apply(lambda x : 10 if x <= 31 else 11 )
df_copy['s'] = 0
df_copy['ds'] = pd.to_datetime(dict(year=df_copy.year, month=df_copy.month, day=df_copy.day_, hour=df_copy.h, minute=df_copy.m))

In [ ]:
# remove columns that are not be used in the model
df_copy.drop(columns=['day','h','m', 's', 'dow', 'year', 'day_', 's', 'month'], inplace=True)

In [ ]:
# fbprophet expect the y column
df_copy.rename(columns={'demand':'y'}, inplace=True)

In [ ]:
df_copy.head()

In [ ]:
#df_copy.groupby('geo_encoded').agg({'ds':'count'}).sort_values(by='ds', ascending=False).head()

In [ ]:
#df_copy = df_copy.sample(frac=0.1, random_state=1)
# we'll take only one of the geohash area as taking entire dataset is extremely expensive
# It would take 1h 28min 24s for training
# so to save time I'll pick first 5 top areas
df_copy = df_copy[df_copy['geo_encoded'].isin([232,261,215,275,507])]

In [ ]:
#sort by datetime
df_copy = df_copy.sort_values(by=['ds'])

In [ ]:
split_n = int(0.8*len(df_copy))
train = df_copy.iloc[:split_n,:]
test = df_copy.iloc[split_n:,:]

In [ ]:
# mcmc_samples=300, changepoint_prior_scale=0.01 (enable this to get uncertainty bound)
m = Prophet(seasonality_mode='multiplicative', \
            weekly_seasonality=True, \
            daily_seasonality=True)

In [ ]:
%%time
m.add_regressor('geo_encoded', mode='multiplicative')
m.fit(train)

In [ ]:
%%time
#future = m.make_future_dataframe(periods=5, freq='min')
future = train[['ds','geo_encoded']]
forecast = m.predict(future)
rmse = np.sqrt(mean_squared_error(train['y'], forecast['yhat']))
print(rmse)

In [ ]:
%%time
#future = m.make_future_dataframe(periods=5, freq='min')
future = test[['ds','geo_encoded']]
forecast = m.predict(future)
rmse = np.sqrt(mean_squared_error(test['y'], forecast['yhat']))
print(rmse)

In [ ]:
future = df_copy[['ds','geo_encoded']]
forecast = m.predict(future)

In [ ]:
forecast['y'] = df_copy['y'].values
forecast['yhat_lower'] =  np.clip(forecast.yhat_lower, 0, 1)
forecast['yhat_upper'] =  np.clip(forecast.yhat_upper, 0, 1)

In [ ]:
f, ax = plt.subplots(figsize=(14, 8))
first = forecast.iloc[:split_n,:]
ax.plot(first.ds, first.y, 'ko', markersize=3)
ax.plot(first.ds, first.yhat, color='steelblue', lw=0.5)
#ax.fill_between(first.ds, first.yhat_lower, first.yhat_upper, color='steelblue', alpha=0.3)

second = forecast.iloc[split_n:,:]
ax.plot(second.ds, second.y, 'ro', markersize=3)
ax.plot(second.ds, second.yhat, color='coral', lw=0.5)
#ax.fill_between(second.ds, second.yhat_lower, second.yhat_upper, color='coral', alpha=0.3)
ax.axvline(str(second.iloc[0]['ds']), color='0.8', alpha=0.9)
ax.grid(ls=':', lw=0.5)

In [ ]:
# clean up 
del df_copy
del forecast
del future
gc.collect()

# Neural Network (LSTM)

We'll try to build a Encoder-Decoder LSTM model for multi-step forecasting with multivariate input data using methods explained in [machinelearningmastery.com](https://machinelearningmastery.com/how-to-develop-lstm-models-for-multi-step-time-series-forecasting-of-household-power-consumption/)

The final average RMSE is significantly lower than fbprohet. Thought I didn't have time to verify the result. I'll leave it to later time.


In [ ]:
# We'll prototype using 100k sample since executing full 4 millions dataset will slow down our step
# once we have good model we can gradually increase the size and observe the accuracy.
df_sample = df.sample(100000, random_state=1)

In [ ]:
df_sample.info()

In [ ]:
df_sample = df_sample.sort_values(by=['day','h','m'])

In [ ]:
# helper for splitting data into train and test
def split_dataset(data, n_step=5):
    n_train = int(0.8 * len(data))
    train, test = data[:n_train,:], data[n_train:,:]
    train = array(split(train, len(train)/n_step))
    test = array(split(test, len(test)/n_step))
    return train, test

In [ ]:
def build_model(train, n_input, batch_size=100, epochs=50, verbose=1):
    # prepare data
    train_x, train_y = to_supervised(train, n_input)
    n_timesteps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]
    # reshape output into [samples, timesteps, features]
    train_y = train_y.reshape((train_y.shape[0], train_y.shape[1], 1))
    # define model
    model = Sequential()
    model.add(LSTM(200, activation='relu', input_shape=(n_timesteps, n_features)))
    model.add(RepeatVector(n_outputs))
    model.add(LSTM(200, activation='relu', return_sequences=True))
    model.add(TimeDistributed(Dense(100, activation='relu')))
    model.add(TimeDistributed(Dense(1)))
    model.compile(loss='mse', optimizer='adam')
    # fit network
    model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, verbose=verbose)
    return model

In [ ]:
def to_supervised(train, n_input, n_out=5):
    # flatten data
    data = train.reshape((train.shape[0]*train.shape[1], train.shape[2]))
    X, y = list(), list()
    in_start = 0
    #step over the entire history one time step at a time
    for _ in range(len(data)):
       # define the end of the input sequence
       in_end = in_start + n_input
       out_end = in_end + n_out
       # ensure we have enough data for this instance
       if out_end < len(data):
          X.append(data[in_start:in_end, :])
          y.append(data[in_end:out_end, 0])
       # move along one time step
       in_start += 1
    return array(X), array(y)

In [ ]:
def forecast(model, history, n_input):
   # flatten data
   data = array(history)
   data = data.reshape((data.shape[0]*data.shape[1], data.shape[2]))
   # retrieve last observations for input data
   input_x = data[-n_input:, :]
   # reshape into [1, n_input, n]
   input_x = input_x.reshape((1, input_x.shape[0], input_x.shape[1]))
   # forecast the next step
   yhat = model.predict(input_x, verbose=0)
   # we only want the vector forecast
   yhat = yhat[0]
   return yhat

In [ ]:
# evaluate one or more forecasts against expected values
def evaluate_forecasts(actual, predicted):
   scores = list()
   # calculate an RMSE score for each day
   for i in range(actual.shape[1]):
      # calculate mse
      mse = mean_squared_error(actual[:, i], predicted[:, i])
      # calculate rmse
      rmse = sqrt(mse)
      # store
      scores.append(rmse)
   # calculate overall RMSE
   s = 0
   for row in range(actual.shape[0]):
      for col in range(actual.shape[1]):
         s += (actual[row, col] - predicted[row, col])**2
   score = sqrt(s / (actual.shape[0] * actual.shape[1]))
   return score, scores

In [ ]:
# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
df_sample[['day','h','m','dow','geo_encoded']] = scaler.fit_transform(df_sample[['day','h','m','dow','geo_encoded']])

In [ ]:
# split into train and test
train, test = split_dataset(df_sample.values)

In [ ]:
train.shape, test.shape

In [ ]:
%%time
n_input = 16 # the lagging 
# fit model
model = build_model(train, n_input)

In [ ]:
%%time
# history is a list of weekly data
history = [x for x in train]
# walk-forward validation over each steps
predictions = list()
for i in range(len(test)):
  # predict the week
  yhat_sequence = forecast(model, history, n_input)
  # store the predictions
  predictions.append(yhat_sequence)
  # get real observation and add to history for predicting the next week
  history.append(test[i, :])
# evaluate predictions days for each week
predictions = array(predictions)

In [ ]:
predictions.shape

In [ ]:
score, scores = evaluate_forecasts(test[:, :, 0], predictions)

In [ ]:
s_scores = ', '.join(['%.1f' % s for s in scores])
print('lstm: [%.3f] %s' % (score, s_scores))

In [ ]:
steps = [1,2,3,4,5]
plt.plot(steps, scores, marker='o', label='lstm')
plt.show()